In [78]:
!pip install openpyxl

You should consider upgrading via the '/Users/rema/.virtualenvs/ds/bin/python -m pip install --upgrade pip' command.


In [79]:
import pandas as pd
import json
import requests

# Структура данных
### ID - уникальный идентификатор
### Longitude - долгота
### Latitude - широта
### Name - названия
### AdmArea - административный округ
### District - район
### Address - улица и дом

In [80]:
df_columns = ['Name', 'AdmArea', 'District', 'Address', 'geoData', 'Category', 'SubCategory']
df = pd.DataFrame(columns=df_columns)

In [81]:
def add_subcategory(df, sub_category_name):
    df['SubCategory'] = sub_category_name
    return df

def add_category(df, category_name):
    df['Category'] = category_name
    return df

def parse_coordinates(df):
    def parse(row):
        # Словарь
        if type(row['geoData']) == dict:
            geoData = row['geoData']
        # Лист с 2-мя координатами
        elif type(row['geoData']) == list and len(row['geoData']) == 2:
            return row['geoData']
        # Возможно словарь
        elif type(row['geoData']) == str:
            geoData = json.loads(row['geoData'])            
        
        if len(geoData['coordinates']) == 2 and geoData['type'] == 'Point':
            return geoData['coordinates']
        elif geoData['type'] == 'MultiPoint':
            return geoData['coordinates'][0]
        elif geoData['type'] == 'Polygon':
            pass

    df['geoData'] = df[['geoData']].apply(parse, axis=1)
    return df

def get_district_and_admarea(df):
    def get_data(row):
        if pd.isna(row['AdmArea']) or pd.isna(row['District']) or pd.isna(row['Address']):
            locate = sorted(row['geoData'], reverse=True)
            locate = f'{locate[0]}, {locate[1]}'
            while True:
                try:
                    r = requests.post('https://geocode.xyz/', data={'locate': locate, 'geoit': "JSON"})
                    data = json.loads(r._content, encoding='cp1251')
                    if 'adminareas' in data:
#                         print(data)
                        print('before', row['AdmArea'], row['District'], row['Address'], locate)
                        if 'admin5' in data['adminareas']:
                            row['AdmArea'] = data['adminareas']['admin5']['name']
                        elif 'admin6' in data['adminareas']:
                            row['AdmArea'] = data['adminareas']['admin6']['name']
                        row['District'] = data['osmtags']['wikipedia'][3:]
#                         if 'name_ru' in data['osmtags']:
#                             row['District'] = data['osmtags']['name_ru']
#                         else:
#                             row['District'] = data['osmtags']['name']
                        row['Address'] = data['staddress']
                        print('after ', row['AdmArea'], row['District'], row['Address'])
                    else:
                        print('ups!!!')
                        raise LookupError
                    return row

                except KeyError as e:
                    print(e, '#######################################')
                    raise KeyError
                except LookupError:
                    continue
        else:
            return row
    df = df.apply(get_data, axis=1)
    return df

# Религия

In [82]:
religion_rpc = pd.read_json('../data/Religion/Религиозные объекты РПЦ.json', encoding="cp1251")
religion_catholics = pd.read_json('../data/Religion/Католические храмы.json', encoding="cp1251")
religion_mosques = pd.read_json('../data/Religion/Мечети.json', encoding="cp1251")
religion_monasteries = pd.read_json('../data/Religion/Монастыри.json', encoding="cp1251")
religion_synagogues = pd.read_json('../data/Religion/Синагоги.json', encoding="cp1251")

religion_df = pd.DataFrame(columns=df_columns)

In [83]:
religion_rpc = add_subcategory(religion_rpc, 'Объект РПЦ')
religion_catholics = add_subcategory(religion_catholics, 'Католический храм')
religion_mosques = add_subcategory(religion_mosques, 'Мечеть')
religion_monasteries = add_subcategory(religion_monasteries, 'Монастырь')
religion_synagogues = add_subcategory(religion_synagogues, 'Синагога')

In [84]:
columns = ['Name', 'AdmArea', 'District', 'Address', 'geoData', 'SubCategory']
religion_df = religion_df.append(religion_rpc[columns])

In [85]:
religion_catholics.rename(columns={"ObjectName": "Name"}, inplace=True)
religion_df = religion_df.append(religion_catholics[columns])

In [86]:
religion_mosques.rename(columns={"ObjectName": "Name"}, inplace=True)
religion_df = religion_df.append(religion_mosques[columns])

In [87]:
religion_monasteries.rename(columns={"NameOfReligiousOrganization": "Name"}, inplace=True)
religion_df = religion_df.append(religion_monasteries[columns])

In [88]:
religion_synagogues.rename(columns={"ObjectName": "Name"}, inplace=True)
religion_df = religion_df.append(religion_synagogues[columns])

In [89]:
religion_df = add_category(religion_df, 'Религия')

In [90]:
religion_df = parse_coordinates(religion_df)

In [91]:
df = df.append(religion_df)

# Культура

In [92]:
сulture_objects = pd.read_json('../data/Culture/Объекты культурного наследия.json', encoding="cp1251")
сulture_museums = pd.read_json('../data/Culture/Музеи.json', encoding="cp1251")

In [93]:
сulture_objects.drop(['geoData'], axis=1, inplace=True)
сulture_objects.rename(columns={"ObjectNameOnDoc": "Name",
                                'Addresses': 'Address',
                                'geodata_center': 'geoData'}, inplace=True)
сulture_objects = parse_coordinates(сulture_objects)

In [94]:
сulture_objects = add_category(сulture_objects, 'Культура')
сulture_objects = add_subcategory(сulture_objects, 'Объект культурного наследия')

In [95]:
df = df.append(сulture_objects[df_columns])

In [96]:
сulture_museums.rename(columns={"FullName": "Name"}, inplace=True)
сulture_museums = parse_coordinates(сulture_museums)

сulture_museums['Address'] = сulture_museums[['ObjectAddress']].apply(
    lambda row: row['ObjectAddress'][0]['Address'], axis=1)
сulture_museums['District'] = сulture_museums[['ObjectAddress']].apply(
    lambda row: row['ObjectAddress'][0]['District'], axis=1)
сulture_museums['AdmArea'] = сulture_museums[['ObjectAddress']].apply(
    lambda row: row['ObjectAddress'][0]['AdmArea'], axis=1)

сulture_museums = add_category(сulture_museums, 'Культура')
сulture_museums = add_subcategory(сulture_museums, 'Музей')

In [97]:
df = df.append(сulture_museums[df_columns])

# Спорт

In [98]:
sport_objects = pd.read_json('../data/Sport/sportivnye-obekty-goroda-moskvy.json', encoding="cp1251")

In [99]:
sport_objects.rename(columns={"FullName": "Name"}, inplace=True)
sport_objects['Address'] = sport_objects[['ObjectAddress']].apply(
    lambda row: row['ObjectAddress'][0]['Address'], axis=1)
sport_objects['District'] = sport_objects[['ObjectAddress']].apply(
    lambda row: row['ObjectAddress'][0].get('District', pd.NA), axis=1)
sport_objects['AdmArea'] = sport_objects[['ObjectAddress']].apply(
    lambda row: row['ObjectAddress'][0].get('AdmArea', pd.NA), axis=1)

sport_objects = add_category(sport_objects, 'Спорт')
sport_objects = add_subcategory(sport_objects, 'Спортивное сооружение')

sport_objects.drop(['geoData'], axis=1, inplace=True)
sport_objects.rename(columns={'geodata_center': 'geoData'}, inplace=True)
sport_objects = parse_coordinates(sport_objects)

In [100]:
df = df.append(sport_objects[df_columns])

# Транспорт

In [101]:
transport_subway_exits = pd.read_excel(
    '../data/Transport/Входы и выходы вестибюлей станций Московского метрополитена.xlsx')

transport_bus_stops = pd.read_json(
    '../data/Transport/Маршруты и остановки наземного городского пассажирского транспорта.json', encoding="cp1251")

/Users/rema/.virtualenvs/ds/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [102]:
transport_subway_exits["Address"] = pd.NA
transport_subway_exits = add_category(transport_subway_exits, 'Транспорт')
transport_subway_exits = add_subcategory(transport_subway_exits, 'Выход из метро')

In [103]:
transport_subway_exits.rename(columns={'geodata_center': 'geoData'}, inplace=True)

In [104]:
transport_subway_exits = parse_coordinates(transport_subway_exits)

In [105]:
df = df.append(transport_subway_exits[df_columns])

In [106]:
transport_bus_stops = parse_coordinates(transport_bus_stops)
transport_bus_stops["Address"] = pd.NA
transport_bus_stops["District"] = pd.NA
transport_bus_stops["AdmArea"] = pd.NA
transport_bus_stops.rename(columns={'stop_name': 'Name'}, inplace=True)
transport_bus_stops = add_category(transport_bus_stops, 'Транспорт')
transport_bus_stops = add_subcategory(transport_bus_stops, 'Остановки наземного пассажирского транспорта')

In [107]:
df = df.append(transport_bus_stops[df_columns])

In [108]:
df.tail()

,Name,AdmArea,District,Address,geoData,Category,SubCategory
12192,Музыкальная школа,<NA>,<NA>,<NA>,"[37.71896446, 55.67398777]",Транспорт,Остановки наземного пассажирского транспорта
12193,Станция Крюково,<NA>,<NA>,<NA>,"[37.17474711, 55.98088799]",Транспорт,Остановки наземного пассажирского транспорта
12194,Станция МЦК «Ростокино»,<NA>,<NA>,<NA>,"[37.66528152, 55.8428194]",Транспорт,Остановки наземного пассажирского транспорта
12195,Метро «Университет»,<NA>,<NA>,<NA>,"[37.53513175, 55.69367601]",Транспорт,Остановки наземного пассажирского транспорта
12196,Метро «Каширская»,<NA>,<NA>,<NA>,"[37.65150982, 55.6553112]",Транспорт,Остановки наземного пассажирского транспорта


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22500 entries, 0 to 12196
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         22500 non-null  object
 1   AdmArea      10284 non-null  object
 2   District     10284 non-null  object
 3   Address      9283 non-null   object
 4   geoData      22500 non-null  object
 5   Category     22500 non-null  object
 6   SubCategory  22500 non-null  object
dtypes: object(7)
memory usage: 1.4+ MB


In [ ]:
import datetime
start = datetime.datetime.now()
df = get_district_and_admarea(df)
end = datetime.datetime.now()
print(end - start)

<ipython-input-81-1608110b09a5>:39: DeprecationWarning: 'encoding' is ignored and deprecated. It will be removed in Python 3.9
  data = json.loads(r._content, encoding='cp1251')


before <NA> <NA> обл. Московская, 0,  , 55.8919654827986, 38.2393480645009
after  городской округ Лосино-Петровский Лосино-Петровский {}
ups!!!
before <NA> <NA> обл. Московская, 0,  , 55.9763370222124, 37.5435902140592
after  городской округ Мытищи Городской округ Мытищи Александровская улица
ups!!!
ups!!!
before <NA> <NA> обл. Московская, 0,  , 55.1314816376423, 37.5239186134031
after  городской округ Чехов Чеховский район {}
before <NA> <NA> обл. Московская, 0,  , 55.1302294523401, 37.5253015549028
after  городской округ Чехов Чеховский район Промышленная улица
before <NA> <NA> обл. Московская, г. Мытищи, д. Новоалександрово, строение 1 55.9765025937976, 37.545373905063
after  городской округ Мытищи Городской округ Мытищи Александровская улица
before <NA> <NA> обл. Московская, г. Химки, д. Голиково, снт. Зеленый Бор, корп. 1 55.9338159296379, 37.2825583816087
after  городской округ Химки Городской округ Химки Красная улица
before <NA> <NA> обл. Московская, 0,  , 56.0550424102821, 36.

before Северный административный округ Войковский район <NA> 55.839091, 37.487262
after  Северный административный округ Войковский район улица Адмирала Макарова
ups!!!
before Северный административный округ Войковский район <NA> 55.8198862, 37.4966084
after  Северный административный округ Войковский район 3-й Войковский проезд
before Северный административный округ Войковский район <NA> 55.8200536, 37.4978475
after  Северный административный округ Войковский район площадь Ганецкого
before Северо-Восточный административный округ район Отрадное <NA> 55.8483448, 37.5903673
after  Северо-Восточный административный округ Отрадное (район Москвы) Сигнальный проезд
before Северный административный округ Головинский район <NA> 55.8405471, 37.4861169
after  Северный административный округ Головинский район Кронштадтский бульвар
before Северный административный округ Войковский район <NA> 55.8180816, 37.4980328
after  Северный административный округ Войковский район Улица Зои и Александра Космо

before Южный административный округ район Орехово-Борисово Южное <NA> 55.610249, 37.716894
after  Южный административный округ Орехово-Борисово Северное Каширское шоссе
before Южный административный округ район Орехово-Борисово Южное <NA> 55.6099729, 37.7175836
after  Южный административный округ Орехово-Борисово Южное Каширское шоссе
before Южный административный округ район Орехово-Борисово Южное <NA> 55.610893, 37.720888
after  Южный административный округ Орехово-Борисово Южное Ореховый бульвар
before Северный административный округ Савёловский район <NA> 55.807779, 37.580614
after  Северный административный округ Бутырский район Бутырская улица
before Северо-Восточный административный округ Бутырский район <NA> 55.8081488, 37.5818692
after  Северный административный округ Бутырский район Бутырская улица
ups!!!
ups!!!
ups!!!
before Юго-Восточный административный округ Южнопортовый район <NA> 55.7178288, 37.6762283
after  Юго-Восточный административный округ Южнопортовый район Метро

before Северо-Западный административный округ район Щукино <NA> 55.7939801, 37.4922446
after  Северный административный округ Щукино (район Москвы) улица Маршала Бирюзова
before Северо-Западный административный округ район Щукино <NA> 55.7941438, 37.4919278
after  Северный административный округ Щукино (район Москвы) улица Маршала Бирюзова
before Северо-Западный административный округ район Щукино <NA> 55.7941853, 37.4925294
after  Северный административный округ Щукино (район Москвы) улица Маршала Бирюзова
before Северо-Западный административный округ район Щукино <NA> 55.7943253, 37.4922657
after  Северный административный округ Щукино (район Москвы) улица Маршала Бирюзова
before Юго-Западный административный округ район Черёмушки <NA> 55.671056, 37.55586
after  Юго-Западный административный округ Черёмушки (район Москвы) Станция метро «Новые Черёмушки»
ups!!!
ups!!!
before Северо-Западный административный округ район Щукино <NA> 55.7928291, 37.4944062
after  Северный административны

...

ups!!!
ups!!!
before Южный административный округ район Нагатино-Садовники <NA> 55.6838692, 37.623651
after  Южный административный округ Нагатино-Садовники Варшавское шоссе
before Центральный административный округ Тверской район <NA> 55.7660404, 37.6081884
after  Центральный административный округ Тверской район (Москва) Страстной бульвар
before Центральный административный округ Тверской район <NA> 55.765875, 37.607782
after  Центральный административный округ Тверской район (Москва) Страстной бульвар
before Центральный административный округ Тверской район <NA> 55.765593, 37.608125
after  Центральный административный округ Тверской район (Москва) Страстной бульвар
ups!!!
before Центральный административный округ Тверской район <NA> 55.7775707, 37.5821305
after  Центральный административный округ Тверской район (Москва) площадь Тверская Застава
ups!!!
before Центральный административный округ Тверской район <NA> 55.769542, 37.597155
after  Центральный административный округ Тверской

before Юго-Западный административный округ район Черёмушки <NA> 55.6771381, 37.5616571
after  Юго-Западный административный округ Черёмушки (район Москвы) Профсоюзная улица
before Юго-Западный административный округ район Черёмушки <NA> 55.676938, 37.562204
after  Юго-Западный административный округ Черёмушки (район Москвы) Профсоюзная улица
before Юго-Западный административный округ Академический район <NA> 55.6787501, 37.5634052
after  Юго-Западный административный округ Академический район (Москва) Профсоюзная улица
before Юго-Западный административный округ Академический район <NA> 55.6785501, 37.5639475
after  Юго-Западный административный округ Академический район (Москва) Профсоюзная улица
before Юго-Западный административный округ Академический район <NA> 55.6870202, 37.5722186
after  Юго-Западный административный округ Академический район (Москва) Профсоюзная улица
ups!!!
before Юго-Западный административный округ Академический район <NA> 55.6868261, 37.5729207
after  Юго-Запа

ups!!!
ups!!!
before Восточный административный округ район Преображенское <NA> 55.7963262, 37.7138546
after  Восточный административный округ Преображенское (район Москвы) Краснобогатырская улица
before Восточный административный округ район Преображенское <NA> 55.7962682, 37.7135247
after  Восточный административный округ Преображенское (район Москвы) Преображенская площадь
ups!!!
before Центральный административный округ Тверской район <NA> 55.777624, 37.58613
after  Центральный административный округ Тверской район (Москва) Заставный переулок
ups!!!
ups!!!
ups!!!
ups!!!
before Центральный административный округ Красносельский район <NA> 55.7750909, 37.6565984
after  Центральный административный округ Красносельский район (Москва) Комсомольская площадь
ups!!!
before Центральный административный округ район Хамовники <NA> 55.735313, 37.593328
after  Центральный административный округ Хамовники (район Москвы) Крымская площадь
ups!!!
ups!!!
before Западный административный округ район 

before Юго-Восточный административный округ район Кузьминки <NA> 55.705253, 37.764105
after  Юго-Восточный административный округ Кузьминки (район Москвы) улица Маршала Чуйкова
before Юго-Восточный административный округ район Текстильщики <NA> 55.709554, 37.733535
after  Юго-Восточный административный округ Текстильщики (район Москвы) Волгоградский проспект
ups!!!
ups!!!
before Юго-Восточный административный округ район Печатники <NA> 55.70855, 37.729033
after  Юго-Восточный административный округ Текстильщики (район Москвы) Люблинская улица
ups!!!
ups!!!
before Юго-Восточный административный округ Рязанский район <NA> 55.717709, 37.793673
after  Юго-Восточный административный округ Рязанский район (Москва) Рязанский проспект
before Юго-Восточный административный округ Южнопортовый район <NA> 55.725542, 37.685442
after  Юго-Восточный административный округ Южнопортовый район St. m. Volgogradskii prospekt
before Северо-Западный административный округ район Северное Тушино <NA> 55.86178

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [ ]:
r = requests.post('https://geocode.xyz/', data={'locate': '55.682188, 37.621783', 'geoit': "JSON"})
data=json.loads(r._content, encoding='cp1251')

In [77]:
data

{'statename': {},
 'distance': '0.047',
 'elevation': '137',
 'osmtags': {'wikipedia': 'ru:Нагорный район (Москва)',
  'wikidata': 'Q1963134',
  'official_status': 'ru:внутригородская территория города федерального значения',
  'name': 'Нагорный район',
  'description': 'район — административно-территориальная единица; муниципальный округ — тип муниципального образования (с 1 июля 2012)',
  'name_ru': 'Нагорный район',
  'name_be': 'Нагорны',
  'admin_level': '8',
  'boundary': 'administrative',
  'type': 'boundary',
  'alt_name': 'муниципальный округ Нагорный',
  'name_en': 'Nagorny District'},
 'state': {},
 'latt': '55.68226',
 'city': 'Nagornyy',
 'prov': 'RU',
 'geocode': 'MOSCOW-IWWFO',
 'geonumber': '8510990127576',
 'country': 'Russian Federation',
 'stnumber': '1',
 'staddress': 'Nizhnie Kotly',
 'inlatt': '55.68219',
 'alt': {'loc': [{'staddress': 'Nizhnie Kotly',
    'stnumber': '1',
    'postal': {},
    'latt': '55.68226',
    'city': 'Nagornyy',
    'prov': {},
    'longt